In [15]:
#Importing Pandas and Numpy Libraries
import pandas as pd
import numpy as np

In [16]:
#Read the Datafile in csv format
df = pd.read_csv('C:\\Users\\Farooq.Ahmed\\Desktop\\Python Assignment\\Movie-data.csv',names=['user_id', 'movie_id', 'rating'])
df.head()

,user_id,movie_id,rating
0,196,242,3
1,186,302,3
2,22,377,1
3,244,51,2
4,166,346,1


In [17]:
#to get the statistics data
df.describe()

,user_id,movie_id,rating
count,100000.00000,100000.000000,100000.000000
mean,462.48475,425.530130,3.529860
std,266.61442,330.798356,1.125674
min,1.00000,1.000000,1.000000
25%,254.00000,175.000000,3.000000
50%,447.00000,322.000000,4.000000
75%,682.00000,631.000000,4.000000
max,943.00000,1682.000000,5.000000


In [18]:
#To get the information about the Dataframe(like columns,column datatypes)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   user_id   100000 non-null  int64
 1   movie_id  100000 non-null  int64
 2   rating    100000 non-null  int64
dtypes: int64(3)
memory usage: 2.3 MB


In [19]:
#To summarize the data in the two dimensional table using pivot_table
user_ratings=df.pivot_table(index=['user_id'],columns=['movie_id'],values='rating')
user_ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
#Splitting the train and test data with 25% as test data and 75% with the Train data
#To develop the model on the 75% of Train data and test it on remaining data 
from sklearn.model_selection import train_test_split
n_users = df.user_id.nunique() #to get the unique user_ids
n_movies = df.movie_id.nunique() #to get the unique mmovie_ids
train_data, test_data = train_test_split(df, test_size=0.25) #here test_size=0.25 for splitting 25% data as test
print(n_users)
print(n_movies)
print(train_data.head())
print(test_data.head())

943
1682
       user_id  movie_id  rating
18652      417       425       4
32199      379       205       5
40500       54       307       4
16677      421       124       4
49711      268       655       4
       user_id  movie_id  rating
10035      312       131       5
74834      830       172       5
19678       49       219       1
40933      313       215       4
8652       256      1040       3


In [22]:
#train_data_matrix is a user-movie matrix, where each row represents a user,column represents a movie, and the cell values contain the ratings given by users to movies
train_data_matrix = np.zeros((n_users, n_movies))
for i in train_data.itertuples():
    #[user_id index, movie_id index] = rating
    train_data_matrix[i[1]-1, i[2]-1] = i[3] 
train_data_matrix

array([[5., 3., 4., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 5., 0., ..., 0., 0., 0.]])

In [24]:
#test_data_matrix is prepared similarly as the train_data_matrix
test_data_matrix = np.zeros((n_users, n_movies))
for i in test_data.itertuples():
    #[user_id index, movie_id index] = rating
    test_data_matrix[i[1]-1, i[2]-1] = i[3]
test_data_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [5., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [25]:
#pairwise_distances function is used to compute user similarity using the cosine distance metric
from sklearn.metrics import pairwise_distances

#The 'cosine' metric is used to compute the cosine similarity between each pair of users.
#user_similarity calculates the pairwise cosine distances between users.
user_similarity = pairwise_distances(train_data_matrix, metric='cosine')

#To calculate movie similarity based on the cosine distance metric
movie_similarity = pairwise_distances(train_data_matrix.T, metric='cosine')

mean_user_rating = train_data_matrix.mean(axis=1)[:, np.newaxis] 
ratings_diff = (train_data_matrix - mean_user_rating) 

user_pred = mean_user_rating + user_similarity.dot(ratings_diff) / np.array([np.abs(user_similarity).sum(axis=1)]).T
#user_pred to compute the predicted ratings for each user using the user-based collaborative filtering approach
user_pred

array([[ 1.56170678,  0.54187871,  0.44801738, ...,  0.25565736,
         0.258073  ,  0.25782814],
       [ 1.31817391,  0.29460657,  0.15646545, ..., -0.06437471,
        -0.06109543, -0.06108123],
       [ 1.33458355,  0.25146204,  0.12019694, ..., -0.10543043,
        -0.10227848, -0.10204221],
       ...,
       [ 1.24350013,  0.21228157,  0.07940437, ..., -0.13264096,
        -0.12952609, -0.12954215],
       [ 1.37823798,  0.31723193,  0.20888266, ..., -0.00974187,
        -0.00678297, -0.00660365],
       [ 1.43907254,  0.40147299,  0.31868419, ...,  0.1243311 ,
         0.12672719,  0.12675657]])

In [26]:
#To make movie rating predictions for users
movie_pred = train_data_matrix.dot(movie_similarity) / np.array([np.abs(movie_similarity).sum(axis=1)])
movie_pred

array([[0.33939871, 0.34691641, 0.36217334, ..., 0.40749554, 0.40014101,
        0.39610808],
       [0.08875249, 0.10301836, 0.09935855, ..., 0.10469958, 0.10467667,
        0.10456862],
       [0.06165773, 0.06568979, 0.06264014, ..., 0.06424747, 0.06367671,
        0.06501469],
       ...,
       [0.02682097, 0.03154825, 0.02958414, ..., 0.03450327, 0.03415169,
        0.03384478],
       [0.12977301, 0.13667927, 0.14419818, ..., 0.15110054, 0.14987665,
        0.15099173],
       [0.22338249, 0.21652703, 0.24093725, ..., 0.27662106, 0.27016557,
        0.2698709 ]])

In [27]:
#the root mean squared error (RMSE) between two sets of ratings: the predicted ratings (pred) and the actual ratings (test)
#to evaluate the performance of recommendation systems or predictive models we have used RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(pred, test):
    pred = pred[test.nonzero()].flatten() 
    test = test[test.nonzero()].flatten()
    return sqrt(mean_squared_error(pred, test))

In [28]:
#This will calculate the RMSE between the user predicted ratings and the actual ratings
rmse(user_pred, test_data_matrix)

3.1239172090812835

In [29]:
#This will calculate the RMSE between the movie predicted ratings and the actual ratings
rmse(movie_pred, test_data_matrix)

3.4524541151517436